In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

from virtualfermlab.prediction.features import (
    build_features,
    SCENARIO_DESCRIPTIONS,
)
from virtualfermlab.prediction.scenarios import (
    load_datasets,
    cross_ratio_split,
)
from virtualfermlab.prediction.evaluation import (
    run_scenario,
    compare_scenarios,
)

plt.rcParams.update({
    "figure.dpi": 120,
    "figure.figsize": (10, 6),
    "axes.grid": True,
    "grid.alpha": 0.3,
})
print("Setup complete.")

In [ ]:
df_11, df_21 = load_datasets()

for label, df in [("1:1 ratio", df_11), ("2:1 ratio", df_21)]:
    times = sorted(df["Time (h)"].unique())
    reps = df.groupby("Time (h)").size()
    print(f"\n{'=' * 50}")
    print(f"{label}")
    print(f"  Shape: {df.shape}")
    print(f"  Unique time points ({len(times)}): {times}")
    print(f"  Replicates per time point:")
    for t, n in reps.items():
        print(f"    {t:>8.1f} h : {n} replicates")
    print(f"  OD600 range:   [{df['OD600 (-)'].min():.3f}, {df['OD600 (-)'].max():.3f}]")
    print(f"  Glucose range: [{df['[Glucose] (g/L)'].min():.3f}, {df['[Glucose] (g/L)'].max():.3f}]")
    print(f"  Xylose range:  [{df['[Xylose] (g/L)'].min():.3f}, {df['[Xylose] (g/L)'].max():.3f}]")

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

for col, (label, df, color) in enumerate(
    [("1:1 ratio", df_11, "C0"), ("2:1 ratio", df_21, "C1")]
):
    # Row 0: growth curves
    ax = axes[0, col]
    ax.scatter(df["Time (h)"], df["OD600 (-)"], alpha=0.5, s=20, c=color)
    med = df.groupby("Time (h)").median(numeric_only=True).reset_index()
    ax.plot(med["Time (h)"], med["OD600 (-)"], "o-", c=color, ms=4)
    ax.set_xlabel("Time (h)")
    ax.set_ylabel("OD600")
    ax.set_title(f"{label} — Growth Curve")

    # Row 1: substrate concentrations
    ax = axes[1, col]
    ax.scatter(df["Time (h)"], df["[Glucose] (g/L)"], alpha=0.5, s=20, c="tab:orange", label="Glucose")
    ax.scatter(df["Time (h)"], df["[Xylose] (g/L)"], alpha=0.5, s=20, c="tab:green", label="Xylose")
    ax.plot(med["Time (h)"], med["[Glucose] (g/L)"], "o-", c="tab:orange", ms=4)
    ax.plot(med["Time (h)"], med["[Xylose] (g/L)"], "o-", c="tab:green", ms=4)
    ax.set_xlabel("Time (h)")
    ax.set_ylabel("Concentration (g/L)")
    ax.set_title(f"{label} — Substrate Depletion")
    ax.legend(fontsize=8)

# Right column: OD600 vs substrate scatter (motivation for Scenario 1 ambiguity)
ax = axes[0, 2]
for label, df, marker in [("1:1", df_11, "o"), ("2:1", df_21, "^")]:
    ax.scatter(df["[Biomass] (g/L)"], df["[Glucose] (g/L)"],
               alpha=0.5, s=20, marker=marker, label=f"{label} Glucose")
ax.set_xlabel("Biomass (g/L)")
ax.set_ylabel("Glucose (g/L)")
ax.set_title("Biomass vs Glucose (Scenario 1 ambiguity)")
ax.legend(fontsize=8)

ax = axes[1, 2]
for label, df, marker in [("1:1", df_11, "o"), ("2:1", df_21, "^")]:
    ax.scatter(df["[Biomass] (g/L)"], df["[Xylose] (g/L)"],
               alpha=0.5, s=20, marker=marker, label=f"{label} Xylose")
ax.set_xlabel("Biomass (g/L)")
ax.set_ylabel("Xylose (g/L)")
ax.set_title("Biomass vs Xylose (Scenario 1 ambiguity)")
ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

print("\nKey observation: the same biomass value can correspond to very")
print("different substrate concentrations depending on the fermentation phase")
print("(diauxic growth). This fundamental ambiguity motivates adding temporal")
print("context (Scenarios 2-6).")

In [ ]:
# Demonstrate feature engineering for Scenario 3
X_demo, y_demo = build_features(df_11, scenario=3)
print(f"Scenario 3 features for 1:1 dataset:")
print(f"  Feature matrix shape: {X_demo.shape}")
print(f"  Target matrix shape:  {y_demo.shape}")
print(f"  Feature names ({len(X_demo.columns)}):")
for i, col in enumerate(X_demo.columns, 1):
    print(f"    {i:2d}. {col}")
print(f"\n  NaN count: {X_demo.isna().sum().sum()}")
print(f"\nFirst 3 samples:")
display(X_demo.head(3))
display(y_demo.head(3))


# ── Helper function for running and displaying scenario results ──

all_results = {}       # primary direction: train 1:1 -> test 2:1
all_results_rev = {}   # reverse direction: train 2:1 -> test 1:1


def run_and_display(scenario):
    """Run a scenario, display metrics, plot predictions & feature importance."""
    desc = SCENARIO_DESCRIPTIONS[scenario]
    print("=" * 60)
    print(f"Scenario {scenario}: {desc}")
    print("=" * 60)

    X_tr, y_tr, X_te, y_te = cross_ratio_split(df_11, df_21, scenario)
    print(f"\nTrain (1:1): {X_tr.shape[0]} samples, {X_tr.shape[1]} features")
    print(f"Test  (2:1): {X_te.shape[0]} samples, {X_te.shape[1]} features")

    # NaN check
    assert X_tr.isna().sum().sum() == 0, "NaN in training features!"
    assert X_te.isna().sum().sum() == 0, "NaN in test features!"

    # Run
    res = run_scenario(scenario, X_tr, y_tr, X_te, y_te)
    all_results[scenario] = res

    print("\nMetrics (Train 1:1 -> Test 2:1):")
    display(res["metrics"])

    # ── Prediction vs Actual ──
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    for ax, target in zip(axes, ["glucose", "xylose"]):
        y_true = y_te[target].values
        y_pred = res["predictions"][f"xgb_{target}"]
        ax.scatter(y_true, y_pred, alpha=0.6, edgecolor="k", linewidth=0.5, s=40)
        lo = min(y_true.min(), y_pred.min()) - 1
        hi = max(y_true.max(), y_pred.max()) + 1
        ax.plot([lo, hi], [lo, hi], "r--", alpha=0.5, label="Perfect")
        ax.set_xlabel(f"Actual {target.title()} (g/L)")
        ax.set_ylabel(f"Predicted {target.title()} (g/L)")
        ax.set_title(f"Scenario {scenario}: {target.title()}")
        r2 = res["metrics"].query(
            "model == 'XGBoost' and target == @target"
        )["R2"].values[0]
        ax.annotate(f"R\u00b2 = {r2:.3f}", xy=(0.05, 0.95),
                    xycoords="axes fraction", fontsize=10, va="top")
    plt.tight_layout()
    plt.show()

    # ── Feature Importance ──
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    for ax, target in zip(axes, ["glucose", "xylose"]):
        model = res["models"][f"xgb_{target}"]
        imp = model.feature_importances_
        names = np.array(list(X_tr.columns))
        top_k = min(10, len(names))
        idx = np.argsort(imp)[-top_k:]
        ax.barh(names[idx], imp[idx], color="steelblue")
        ax.set_xlabel("Importance (gain)")
        ax.set_title(f"Scenario {scenario} Top Features - {target.title()}")
    plt.tight_layout()
    plt.show()

    # ── Reverse direction ──
    X_tr_r, y_tr_r, X_te_r, y_te_r = cross_ratio_split(df_21, df_11, scenario)
    res_rev = run_scenario(scenario, X_tr_r, y_tr_r, X_te_r, y_te_r)
    all_results_rev[scenario] = res_rev
    print("\nReverse Metrics (Train 2:1 -> Test 1:1):")
    display(res_rev["metrics"])
    print()

In [ ]:
run_and_display(1)

In [ ]:
run_and_display(2)

In [ ]:
run_and_display(3)

In [ ]:
run_and_display(4)

In [ ]:
run_and_display(5)

In [ ]:
run_and_display(6)

In [ ]:
# ── Summary table ──
summary = compare_scenarios(all_results)
print("=== Scenario Comparison (Train 1:1 -> Test 2:1) ===\n")
display(summary.query("model == 'XGBoost'").reset_index(drop=True))

summary_rev = compare_scenarios(all_results_rev)
print("\n=== Scenario Comparison (Train 2:1 -> Test 1:1) ===\n")
display(summary_rev.query("model == 'XGBoost'").reset_index(drop=True))

# ── Bar chart: MAE by scenario ──
xgb_fwd = summary.query("model == 'XGBoost'")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
for ax, target in zip(axes, ["glucose", "xylose"]):
    sub = xgb_fwd.query("target == @target").sort_values("scenario")
    x = sub["scenario"].values
    ax.bar(x - 0.15, sub["MAE"].values, width=0.3, label="MAE", color="steelblue")
    ax.bar(x + 0.15, sub["RMSE"].values, width=0.3, label="RMSE", color="coral")
    ax.set_xlabel("Scenario")
    ax.set_ylabel("Error (g/L)")
    ax.set_title(f"{target.title()} — MAE & RMSE by Scenario")
    ax.set_xticks(range(1, 7))
    ax.legend()
plt.tight_layout()
plt.show()

# ── R-squared line plot ──
fig, ax = plt.subplots(figsize=(8, 5))
for target, marker in [("glucose", "o"), ("xylose", "s")]:
    sub = xgb_fwd.query("target == @target").sort_values("scenario")
    ax.plot(sub["scenario"], sub["R2"], f"-{marker}", ms=8, label=target.title())
ax.set_xlabel("Scenario")
ax.set_ylabel("R\u00b2")
ax.set_title("XGBoost R\u00b2 by Scenario (Train 1:1 -> Test 2:1)")
ax.set_xticks(range(1, 7))
ax.axhline(y=0, color="gray", linestyle="--", alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

# ── Model comparison (XGBoost vs Linear vs Naive) for best scenario ──
best_sc = xgb_fwd.groupby("scenario")["MAE"].mean().idxmin()
print(f"\nBest scenario by average MAE: Scenario {best_sc}")
best_metrics = summary.query("scenario == @best_sc")
display(best_metrics)

In [ ]:
# Analyse errors for the best-performing scenario
# Use Scenario 6 as the most-informed scenario
sc = 6
X_tr, y_tr, X_te, y_te = cross_ratio_split(df_11, df_21, sc)
res = all_results[sc]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for col, target in enumerate(["glucose", "xylose"]):
    y_true = y_te[target].values
    y_pred = res["predictions"][f"xgb_{target}"]
    residual = y_pred - y_true
    times = X_te["time"].values

    # Residual vs time
    ax = axes[0, col]
    ax.scatter(times, residual, alpha=0.6, s=30, edgecolor="k", linewidth=0.3)
    ax.axhline(y=0, color="r", linestyle="--", alpha=0.5)
    ax.set_xlabel("Time (h)")
    ax.set_ylabel("Residual (Predicted - Actual) (g/L)")
    ax.set_title(f"Scenario {sc}: {target.title()} Residuals vs Time")

    # Mark diauxic switch region (~65-75 h)
    ax.axvspan(65, 75, alpha=0.1, color="orange", label="Diauxic switch")
    ax.legend(fontsize=8)

    # Time-point MAE
    ax = axes[1, col]
    err_df = pd.DataFrame({"time": times, "abs_error": np.abs(residual)})
    tp_mae = err_df.groupby("time")["abs_error"].mean().reset_index()
    ax.bar(tp_mae["time"], tp_mae["abs_error"], width=2, color="steelblue", alpha=0.7)
    ax.axvspan(65, 75, alpha=0.1, color="orange", label="Diauxic switch")
    ax.set_xlabel("Time (h)")
    ax.set_ylabel("MAE (g/L)")
    ax.set_title(f"Scenario {sc}: {target.title()} MAE by Time Point")
    ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

print("\nDiauxic switch region (~65-75 h) analysis:")
for target in ["glucose", "xylose"]:
    y_true = y_te[target].values
    y_pred = res["predictions"][f"xgb_{target}"]
    times = X_te["time"].values
    mask_switch = (times >= 65) & (times <= 75)
    mask_other = ~mask_switch
    if mask_switch.sum() > 0:
        mae_switch = np.mean(np.abs(y_pred[mask_switch] - y_true[mask_switch]))
        mae_other = np.mean(np.abs(y_pred[mask_other] - y_true[mask_other]))
        print(f"  {target.title()}: MAE at switch = {mae_switch:.3f} g/L, "
              f"MAE elsewhere = {mae_other:.3f} g/L")

In [ ]:
# Compare top features across scenarios (glucose model)
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for idx, sc in enumerate(range(1, 7)):
    ax = axes[idx // 3, idx % 3]
    res = all_results[sc]
    model = res["models"]["xgb_glucose"]
    X_tr, _, _, _ = cross_ratio_split(df_11, df_21, sc)
    names = np.array(list(X_tr.columns))
    imp = model.feature_importances_
    top_k = min(8, len(names))
    order = np.argsort(imp)[-top_k:]
    ax.barh(names[order], imp[order], color="steelblue")
    ax.set_title(f"Sc {sc}: {SCENARIO_DESCRIPTIONS[sc][:30]}", fontsize=9)
    ax.tick_params(axis="y", labelsize=8)

plt.suptitle("Feature Importance Comparison (Glucose, XGBoost)", fontsize=13, y=1.01)
plt.tight_layout()
plt.show()

# Same for xylose
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for idx, sc in enumerate(range(1, 7)):
    ax = axes[idx // 3, idx % 3]
    res = all_results[sc]
    model = res["models"]["xgb_xylose"]
    X_tr, _, _, _ = cross_ratio_split(df_11, df_21, sc)
    names = np.array(list(X_tr.columns))
    imp = model.feature_importances_
    top_k = min(8, len(names))
    order = np.argsort(imp)[-top_k:]
    ax.barh(names[order], imp[order], color="coral")
    ax.set_title(f"Sc {sc}: {SCENARIO_DESCRIPTIONS[sc][:30]}", fontsize=9)
    ax.tick_params(axis="y", labelsize=8)

plt.suptitle("Feature Importance Comparison (Xylose, XGBoost)", fontsize=13, y=1.01)
plt.tight_layout()
plt.show()

# Practical Recommendations

## HPLC Measurement Reduction

Based on the scenario comparison above, the following guidelines emerge for
reducing destructive HPLC sampling while maintaining predictive accuracy:

1. **Scenario 1 (OD600 only)** provides a lower bound — single-point OD600
   measurements carry fundamental ambiguity due to diauxic growth.

2. **Scenario 2-3 (short OD600 window +/- HPLC history)** offer a practical
   middle ground. Even 2-3 preceding HPLC measurements substantially
   reduce prediction error.

3. **Scenario 5-6 (full OD600 + partial HPLC)** approach the accuracy of
   direct measurement when the model has access to the full growth trajectory
   and at least initial substrate concentrations.

## Minimum OD600 Sampling Frequency

- At least **3 consecutive time points** are needed to capture growth rate
  trends (Scenario 2).
- Higher sampling frequency around the **diauxic switch point** (~65-75 h)
  improves prediction accuracy, as this is where substrate concentrations
  change most rapidly.

## Deployment Strategy

For a new fermentation run:
1. Measure OD600 continuously (non-destructive, cheap).
2. Take HPLC samples at the **first 2-3 time points** to anchor the model.
3. Use Scenario 3 or 6 predictions for subsequent time points.
4. Validate with occasional HPLC spot-checks at critical phases.

This approach can potentially reduce HPLC sampling by **50-70%** while
maintaining substrate concentration estimates within the model's reported
MAE bounds.